#TASK: Leverage NLP for Tableau formulas identification

Leveraging Natural Language Processing (NLP) for Tableau formulas identification involves using language models to interpret and understand natural language descriptions of analytical tasks and then automatically generating corresponding Tableau formulas. While this is a challenging task, it's feasible with the help of advanced NLP models. Below are general steps to approach this task:

In [1]:
! pip install opencv-python
! pip install pandas
! pip install nltk
! pip install spacy
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import os
import cv2
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize,wordpunct_tokenize
from google.colab.patches import cv2_imshow
nltk.download('punkt')
import spacy
from spacy import displacy
from spacy.matcher import matcher
from spacy.tokens import span
from spacy.language import Language
from spacy.pipeline import EntityRuler

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Identify data source in Tableau

In [3]:
from google.colab import files
file="formulas.xlsx"
if file not in os.listdir('/content/'):
      files.upload()

Saving tableau_data.csv to tableau_data.csv
Saving formulas.xlsx to formulas.xlsx


In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
#! cd /content/drive/My\ Drive/Colab_2024
#! cp /content/drive/My\ Drive/Colab_2024/*.png /content/
#! ls -all

# TASK: FORMULA CONVERSION

1. Define Use Cases:
Clearly define the use cases for which you want to leverage NLP for Tableau formulas identification. For example, you might want to interpret natural language descriptions and generate Tableau formulas for common tasks like aggregations, calculations, or filtering.

2. Data Collection:
Collect a dataset that includes examples of natural language descriptions paired with their corresponding Tableau formulas. The dataset should cover a variety of scenarios to train a model that can generalize well.

In [6]:

df=pd.read_excel("formulas.xlsx")
print(df.head(5))


                                  Description        Formula (Tableau)
0                         Sum of Sepal Length        SUM(Sepal Length)
1      Average Sepal Width across all entries         AVG(Sepal Width)
2                Find the median Petal Length     MEDIAN(Petal Length)
3  Count the number of records in the dataset                 COUNT(*)
4       Count distinct species in the dataset  COUNT DISTINCT(Species)


In [7]:
df.columns

Index(['Description', 'Formula (Tableau)'], dtype='object')

3. Preprocess the Data:
Clean and preprocess the collected dataset. Tokenize the natural language descriptions and normalize the Tableau formulas to create input-output pairs for training the NLP model.

# Tokenization of the formulas.xlsx dataset

In [8]:
dg_tok=df.apply(lambda x:[wordpunct_tokenize(y) for y in x])

In [9]:
dg_tok=dg_tok.rename(columns=lambda x:"Tokenized " + x)
dg_tok=dg_tok.loc[:,dg_tok.columns[[0,1]]]
dg_tok.rename(columns=lambda x:x.strip(), inplace=True)
print(dg_tok.head(5))


                               Tokenized Description  \
0                           [Sum, of, Sepal, Length]   
1      [Average, Sepal, Width, across, all, entries]   
2                 [Find, the, median, Petal, Length]   
3  [Count, the, number, of, records, in, the, dat...   
4       [Count, distinct, species, in, the, dataset]   

        Tokenized Formula (Tableau)  
0        [SUM, (, Sepal, Length, )]  
1         [AVG, (, Sepal, Width, )]  
2     [MEDIAN, (, Petal, Length, )]  
3                      [COUNT, (*)]  
4  [COUNT, DISTINCT, (, Species, )]  


#description={} with keys=['text','label','vocab','stopwords']

In [10]:
description={}
description['stopwords']=['\\','Iri','Se','C','Sepal','ica','Length',"'s",'.',"'", 's', 'a','(',')','Petal','Distinct',"Width",',',"'","[","]"]
description['text']=[sentence for sentence in df[df.columns[0]]]
description['vocab']=list(set([word  for ii in range(len(dg_tok[dg_tok.columns[0]])) for word  in dg_tok[dg_tok.columns[0]][ii] if word not in description['stopwords']]))
len_desc_vocab=len(description['vocab'])

In [11]:
print(len_desc_vocab,description['vocab'])

129 ['versicolor', 'Maximum', 'from', 'bottom', 'ends', 'Calculate', 'List', '1', '10', 'Lowest', 'Lengths', 'in', 'or', 'virginica', 'total', 'setosa', 'observation', 'smallest', 'least', 'not', 'is', 'Median', 'Number', '%', 'year', 'by', 'points', 'greater', 'types', 'median', 'alphabetically', 'records', 'order', 'Rank', '10th', 'field', 'high', 'and', 'January', 'How', 'Iris', 'Group', 'with', 'assuming', 'Highest', 'only', 'average', 'Count', 'Filter', 'value', '3', 'start', 'low', 'starts', 'Entries', 'exactly', 'all', 'many', 'descending', 'name', 'distinct', 'species', 'ascending', 'occurrences', 'flowers', 'each', 'within', 'highest', 'Setosa', 'flower', '4', 'unique', 'grouped', 'Find', 'middle', 'equal', 'percentile', 'to', 'where', 'found', '5', 'Average', 'Select', 'show', 'top', 'entries', 'recorded', 'What', 'Minimum', 'last', 'both', 'above', 'number', 'less', 'Sum', 'dataset', 'over', 'at', 'Aggregate', 'for', 'of', 'Species', 'how', 'contains', 'starting', 'than', 'T

#formulas={} with keys=['text','label','vocab','stopwords']

In [12]:
formulas={}
formulas['stopwords']=['\\','Iri','Se','C','Sepal','ica','Length',"'s",'.',"'", 's', 'a','(',')','Petal','Distinct','distinct',"Width",',','BY','by','from',"[","]"]
formulas['text']=[sentence for sentence in df[df.columns[1]]]
formulas['vocab']=list(set([word  for ii in range(len(dg_tok[dg_tok.columns[1]])) for word  in dg_tok[dg_tok.columns[1]][ii] if word not in formulas['stopwords']]))
len_formulas_vocab=len(formulas['vocab'])

In [13]:
print(len_formulas_vocab,formulas['vocab'])

79 ['CONTAINS', 'versicolor', 'LEFT', 'STARTS', 'FROM', '-', 'LISTED', '9', '1', 'SORTED', '())', 'AND', '{', 'virginica', 'THEN', 'SUM', 'setosa', '([', 'DATETRUNC', "('", 'IF', 'year', 'ORDER', 'alphabetically', 'END', 'YEAR', '!=', 'TODAY', '],', 'WITH', 'MIN', 'MEDIAN', 'MAX', '")', 'GROUP', 'STARTSWITH', 'RANK', '3', '(*)', 'COUNT', 'ASC', '<=', 'TOP', 'highest', 'Setosa', 'INDEX', '4', '<', '>', 'lowest', 'to', ':', '“', '}', 'COUNTD', '()', '5', 'SORT', 'RANK_PERCENTILE', "',", 'BETWEEN', '"', 'ELSE', 'ENDSWITH', '>=', 'Species', 'FIXED', '2', 'DISTINCT', 'NULL', '=', '0', '”)', '])', 'color', 'Date', 'month', 'AVG', 'DESC']


# Recognition of entities

In [14]:
def get_pos_(data,col_name):
  dd=[]
  gg=[]
  for tok in data[col_name]:

          for ent in nlp(str(list(str(tok).split()))):
              # checking text between punct
              dd.append({ent.text:ent.pos_})
              gg.append({ent.pos_:ent.text})
  return dd,gg
def normalize_formula(text):
    # Add space after function names if missing
    text = re.sub(r"(\w)\(", r"\1 (", text)
    # Ensure there's a space before and after parentheses
    text = re.sub(r"\s*\(\s*", " ( ", text)
    text = re.sub(r"\s*\)\s*", " ) ", text)
    return text

def get_entity(data,col_name):
     dd=[]
     dt=[]
     dt_e={}
     for text in data[col_name]:
          if col_name=='Formula (Tableau)':
               ntext = normalize_formula(text)
          else:
               ntext=text
          doc=nlp(ntext)
          dp=[text]
          dt_e["entities"]=[]
          for ent in doc.ents:
                    dp.append([ent.text,ent.label_])
                    dt_e["entities"].append((ent.start,ent.end,ent.label_))
          dt.append((text,dt_e.items()))
          dd.append(dp)
     return dd,dt

In [15]:
def get_patterns(data,col_name):
     dt,dp=get_pos_(data,col_name)

     pos_=[list(item.keys())[0] for item in dp]
     pos_=list(set(pos_))
     print(dt)
     print(col_name,":",pos_)
     patterns={}
     patterns_head=[]
     for key in pos_:
         patterns.update({key:list(set([d[key] for d in dp if  key==list(d.keys())[0] and d[key] not in data["stopwords"]]))})
     return patterns

#Pipe entities to description

In [16]:
dg_ent={}

In [17]:
p_d=dg_ent['Patterns_Description']=get_patterns(description,"vocab")

[{'[': 'X'}, {"'": 'X'}, {'versicolor': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Maximum': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'from': 'ADP'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'bottom': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'ends': 'VERB'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Calculate': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'List': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NOUN'}, {'1': 'NUM'}, {"'": 'NUM'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NUM'}, {'10': 'NUM'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Lowest': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'Lengths': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'in': 'ADP'}, {"'": 'NOUN'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'or': 'CCONJ'}, {"'": 'PUNCT'}, {']': 'P

In [18]:
nlp = spacy.load('en_core_web_sm')

# Function to create and return an EntityRuler with specified patterns
def create_ruler(patterns, label):
    ruler = EntityRuler(nlp, overwrite_ents=True)
    formatted_patterns = [{"label": label, "pattern": pattern} for pattern in patterns]
    ruler.add_patterns(formatted_patterns)
    return ruler

desc_conj=p_d['CCONJ']+ p_d['SCONJ']
desc_intj=p_d['INTJ']
desc_verb=p_d['VERB'] + p_d['AUX']
desc_num=p_d['NUM']
desc_adv=p_d['ADV']
desc_pron=p_d['PRON'] + p_d['DET']
desc_adp=p_d['ADP']
desc_noun=[x for x in p_d['X'] if x != p_d['X'][1]]+p_d['NOUN'] +['dataset']
desc_adj=[p_d['X'][1]]+[x for x in p_d['ADJ'] if x !='dataset' ]
desc_propn=['Sepal Length','Sepal Width', 'Petal Length', 'Petal Width']+p_d['PROPN']

# Create rulers
ruler_conj = create_ruler(desc_conj,'CONJ')
ruler_intj = create_ruler(desc_intj,'INTJ')
ruler_verb = create_ruler(desc_verb,'VERB')
ruler_num  = create_ruler(desc_num,'NUM')
ruler_adv  = create_ruler(desc_adv,'ADV')
ruler_pron = create_ruler(desc_pron,'PRON')
ruler_adp  = create_ruler(desc_adp,'ADP')
ruler_noun = create_ruler(desc_noun, "NOUN")
ruler_adj = create_ruler(desc_adj, "ADJ")
ruler_propn = create_ruler(desc_propn,"PROPN")

# Create component factories
@Language.component("ruler_conj")
def ruler_conj_component(doc):
    return ruler_conj(doc)

@Language.component("ruler_intj")
def ruler_intj_component(doc):
    return ruler_intj(doc)

@Language.component("ruler_verb")
def ruler_verb_component(doc):
    return ruler_verb(doc)

@Language.component("ruler_num")
def ruler_num_component(doc):
    return ruler_num(doc)

@Language.component("ruler_adv")
def ruler_adv_component(doc):
    return ruler_adv(doc)

@Language.component("ruler_pron")
def ruler_pron_component(doc):
    return ruler_pron(doc)

@Language.component("ruler_adp")
def ruler_adp_component(doc):
    return ruler_adp(doc)

@Language.component("ruler_noun")
def ruler_noun_component(doc):
    return ruler_noun(doc)

@Language.component("ruler_adj")
def ruler_adj_component(doc):
    return ruler_adj(doc)

@Language.component('ruler_propn')
def ruler_propn_component(doc):
     return ruler_propn(doc)

# Add the components to the pipeline
nlp.add_pipe("ruler_intj", before="ner")
nlp.add_pipe("ruler_verb",before="ner")
nlp.add_pipe("ruler_num", before="ner")
nlp.add_pipe("ruler_adv", before="ner")
nlp.add_pipe("ruler_pron", before="ner")
nlp.add_pipe("ruler_adp", before="ner")
nlp.add_pipe("ruler_noun", before="ner")
nlp.add_pipe("ruler_adj", before="ner")
nlp.add_pipe("ruler_propn", before = "ner")

# Test
#doc=nlp("Sum of Sepal Length. Average Sepal Width across all entries . Count the number of records in the dataset")
#for ent in doc.ents:
#    print(ent.text, ent.label_)
ent_desc,Train_desc_data=get_entity(df,df.columns[0])

In [19]:
for pat in p_d:
   print({pat:p_d[pat]})

{'PRON': ['each', 'there', 'this', 'What', 'all', 'both']}
{'ADP': ['within', 'for', 'above', 'by', 'in', 'of', 'from', 'over', 'with', 'at', 'than', 'between', 'across']}
{'PROPN': ['Group', 'Count', 'Filter', 'Maximum', 'Date', 'Setosa', 'January', 'Median', 'Rank', 'Minimum', 'Iris']}
{'AUX': ['are', 'is']}
{'X': ['setosa', '10th', 'median', 'virginica']}
{'SCONJ': ['where', 'how', 'How']}
{'NUM': ['5', '0', '3', '1', '10', '4', '2']}
{'DET': ['the']}
{'INTJ': ['%']}
{'VERB': ['Find', 'contains', 'starting', 'ends', 'Show', 'found', 'assuming', 'show', 'recorded', 'start', 'starts', 'sorted', 'descending', 'Display', 'ascending', 'Organize', 'count', 'Aggregate', 'grouped']}
{'ADV': ['Sort', 'only', 'alphabetically', 'exactly']}
{'PART': ['not', 'to']}
{'PUNCT': ['?']}
{'NOUN': ['field', 'versicolor', 'Species', 'List', 'Lengths', 'entries', 'Widths', 'observation', 'color', 'value', 'number', 'month', 'Number', 'Entries', 'name', 'year', 'species', 'flowers', 'occurrences', 'Sum', 

In [20]:
print(ent_desc)

[['Sum of Sepal Length', ['Sum', 'NOUN'], ['of', 'ADP'], ['Sepal Length', 'PROPN']], ['Average Sepal Width across all entries', ['Average', 'ADJ'], ['Sepal Width', 'PROPN'], ['across', 'ADP'], ['all', 'PRON'], ['entries', 'NOUN']], ['Find the median Petal Length', ['Find', 'VERB'], ['the', 'PRON'], ['median', 'NOUN'], ['Petal Length', 'PROPN']], ['Count the number of records in the dataset', ['Count', 'PROPN'], ['the', 'PRON'], ['number', 'NOUN'], ['of', 'ADP'], ['records', 'NOUN'], ['in', 'ADP'], ['the', 'PRON'], ['dataset', 'NOUN']], ['Count distinct species in the dataset', ['Count', 'PROPN'], ['distinct', 'ADJ'], ['species', 'NOUN'], ['in', 'ADP'], ['the', 'PRON'], ['dataset', 'NOUN']], ['Minimum value of Sepal Length', ['Minimum', 'PROPN'], ['value', 'NOUN'], ['of', 'ADP'], ['Sepal Length', 'PROPN']], ['Maximum Petal Width found in the dataset', ['Maximum', 'PROPN'], ['Petal Width', 'PROPN'], ['found', 'VERB'], ['in', 'ADP'], ['the', 'PRON'], ['dataset', 'NOUN']], ['Group records 

#Train Description Data

In [21]:
for item in Train_desc_data:
    print(item)

('Sum of Sepal Length', dict_items([('entities', [(0, 1, 'NOUN'), (1, 2, 'PRON'), (2, 3, 'PROPN'), (3, 4, 'NOUN'), (5, 6, 'PRON'), (6, 7, 'NOUN'), (7, 8, 'NOUN'), (8, 9, 'VERB'), (9, 10, 'ADP'), (14, 15, 'VERB'), (15, 16, 'ADP')])]))
('Average Sepal Width across all entries', dict_items([('entities', [(0, 1, 'NOUN'), (1, 2, 'PRON'), (2, 3, 'PROPN'), (3, 4, 'NOUN'), (5, 6, 'PRON'), (6, 7, 'NOUN'), (7, 8, 'NOUN'), (8, 9, 'VERB'), (9, 10, 'ADP'), (14, 15, 'VERB'), (15, 16, 'ADP')])]))
('Find the median Petal Length', dict_items([('entities', [(0, 1, 'NOUN'), (1, 2, 'PRON'), (2, 3, 'PROPN'), (3, 4, 'NOUN'), (5, 6, 'PRON'), (6, 7, 'NOUN'), (7, 8, 'NOUN'), (8, 9, 'VERB'), (9, 10, 'ADP'), (14, 15, 'VERB'), (15, 16, 'ADP')])]))
('Count the number of records in the dataset', dict_items([('entities', [(0, 1, 'NOUN'), (1, 2, 'PRON'), (2, 3, 'PROPN'), (3, 4, 'NOUN'), (5, 6, 'PRON'), (6, 7, 'NOUN'), (7, 8, 'NOUN'), (8, 9, 'VERB'), (9, 10, 'ADP'), (14, 15, 'VERB'), (15, 16, 'ADP')])]))
('Count disti

#Test Description Data

#Pipeline Formulas (Tableau) entities

In [22]:
p_t=dg_ent['Patterns_Formula(Tableau)']=get_patterns(formulas,'vocab')

[{'[': 'X'}, {"'": 'PUNCT'}, {'CONTAINS': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'versicolor': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'LEFT': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PROPN'}, {'STARTS': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'FROM': 'ADP'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'INTJ'}, {'-': 'VERB'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'LISTED': 'VERB'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NUM'}, {'9': 'NUM'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NOUN'}, {'1': 'NUM'}, {"'": 'NUM'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'SORTED': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {')': 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'AND': 'CCONJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'{': 'PUNCT'}, 

In [23]:
for pat in p_t:
   print({pat:p_t[pat]})

{'PRON': ['}']}
{'ADP': ['BETWEEN', 'FROM', 'WITH']}
{'PROPN': ['LEFT', 'NULL', 'STARTS', 'DATETRUNC', 'Date', 'Setosa', 'COUNT', 'ASC', 'END', 'SUM', 'MAX', 'GROUP', 'MIN', 'COUNTD', 'AVG', 'MEDIAN']}
{'ADJ': ['highest', 'lowest', 'SORTED']}
{'X': ['CONTAINS', 'STARTSWITH', '<', '=', '>', 'virginica', 'setosa']}
{'AUX': ['RANK_PERCENTILE']}
{'SCONJ': ['IF']}
{'INTJ': []}
{'VERB': ['-', 'LISTED', 'RANK', 'DESC']}
{'ADV': ['THEN', 'SORT', 'ELSE', 'alphabetically']}
{'PART': ['to']}
{'PUNCT': ['*', '"', '!', '{', '”', ':', '“']}
{'NOUN': ['DISTINCT', 'versicolor', 'Species', 'ORDER', 'TODAY', 'month', 'ENDSWITH', 'YEAR', 'INDEX', 'FIXED', 'color', 'year']}
{'CCONJ': ['AND']}
{'NUM': ['5', 'TOP', '0', '3', '9', '1', '4', '2']}


In [24]:
p_t['ARG']=['[Sepal Length]','[Sepal Width]','[Petal Length]','[Petal Width]', '[Species]']
p_t['ARGN']=['Sepal Length','Sepal Width','Petal Length','Petal Width', 'Species']
p_t['NOUN'] =['month', 'TODAY', 'color','YEAR','versicolor','setosa','year','virginica']
p_t['STOPWORDS']=['[Sepal','Sepal','[Petal','Width]','Width','Petal','Length]',"'Se'",':','"virginica",'"setosa",'','{',')','*',"'Setosa'","'month', ",'4.0 AND','"setosa" AND','ica',"Iri",'!= 3.0','"versicolor" AND',"Species STARTS WITH 'Se'","Length",'"Iri"' ]
p_t['NUM']=[ff  for ff in p_t['NUM'] if ff !='TOP']+['1.5','2.5','4.5','3.0','3','2.0','5.0']
p_t['ADP']+=['to','AND','>=','<','!=','<=','=','>']
p_t['ADV']+=['highest to lowest','ASC','DESC','NULL','LEFT']
p_t['ADJ']=[ff for ff in p_t['ADJ'] if  ff!='SORTED']
ptt=p_t['ARG']+p_t['NOUN']+p_t['NUM'] + p_t['STOPWORDS'] + p_t['ARGN'] + p_t['ADV'] + p_t['ADP']+p_t['ADJ']
p_t['OPER']=['STARTS WITH']

In [25]:
print(p_t['OPER'])

['STARTS WITH']


In [26]:
import re
p_t['OPER']=[]
p_t['COPER']=[]
patterns='\(| = | >= | > | \[ | \) | \{ | \]'
tr_0=[]
for tr in df[df.columns[1]]:
    try:
      it0,it1=re.split(patterns,tr)[0:2]
    except:
         it0=re.split(patterns,tr)[0:2][0]

    it01=it0.split()

    if len(it01)==1:
           iten=it01[0]
    elif len(it01)==2:
            iten=it01[0] +' '+ it01[1]
    else:
          iten=''
          for item in it01:
            if item not in tr_0 and item not in ptt:
               iten+=item +' '
          iten=iten.rstrip()
          #print(it01,'-->',iten)

    if iten not in tr_0 and iten not in ptt:
              #print(it01)
              tr_0.append(iten)


print(tr_0)

['SUM', 'AVG', 'MEDIAN', 'COUNT', 'COUNT DISTINCT', 'MIN', 'MAX', 'GROUP BY', 'ORDER BY', 'TOP BY', 'STARTS', 'COUNTD', 'ENDSWITH', 'INDEX', 'IF', 'DATETRUNC', '[Date]', 'RANK_PERCENTILE', 'FIXED', 'SORTED BY', 'RANK', 'LISTED', 'STARTSWITH']


In [27]:
p_t['OPER']+=tr_0
p_t['OPER'][p_t['OPER'].index('LISTED')]='LISTED FROM'
p_t['OPER'][p_t['OPER'].index('TOP BY')]='TOP'
p_t['OPER']+=['IF CONTAINS','BY']
p_t['NOUN']=[ff for ff in p_t['NOUN'] if ff not in p_t['OPER']]
#p_t['PROPN']=[ff for ff in p_t['PROPN'] if ff not in p_t['OPER']+p_t['NOUN']]


# Pipe entities to Formulas Tableau

In [28]:

nlp = spacy.load('en_core_web_sm')

# Function to create and return an EntityRuler with specified patterns

oper_arg=p_t['ARG']
oper_argn=p_t['ARGN']
oper_num=p_t['NUM']
oper_adv=p_t['ADV']
oper_oper=p_t['OPER']
oper_adp=p_t['ADP']
oper_noun=p_t['NOUN']
oper_adj=p_t['ADJ']

# Create rulers
ruler_arg = create_ruler(oper_arg,'ARG')
#ruler_intj = create_ruler(oper_intj,'INTJ')
ruler_argn = create_ruler(oper_argn,'ARGN')
ruler_num  = create_ruler(oper_num,'NUM')
ruler_adv  = create_ruler(oper_adv,'ADV')
#ruler_pron = create_ruler(oper_pron,'PRON')
ruler_adp  = create_ruler(oper_adp,'ADP')
ruler_noun = create_ruler(oper_noun, "NOUN")
ruler_adj = create_ruler(oper_adj, "ADJ")
ruler_oper = create_ruler(oper_oper,"OPER")

# Create component factories
@Language.component("ruler_arg")
def ruler_arg_component(doc):
    return ruler_arg(doc)

#@Language.component("ruler_intj")
#def ruler_intj_component(doc):
#    return ruler_intj(doc)

@Language.component("ruler_argn")
def ruler_argn_component(doc):
    return ruler_argn(doc)

@Language.component("ruler_num")
def ruler_num_component(doc):
    return ruler_num(doc)

@Language.component("ruler_adv")
def ruler_adv_component(doc):
    return ruler_adv(doc)

@Language.component("ruler_oper")
def ruler_oper_component(doc):
    return ruler_oper(doc)

@Language.component("ruler_adp")
def ruler_adp_component(doc):
    return ruler_adp(doc)

@Language.component("ruler_noun")
def ruler_noun_component(doc):
    return ruler_noun(doc)

@Language.component("ruler_adj")
def ruler_adj_component(doc):
    return ruler_adj(doc)

#@Language.component('ruler_propn')
#def ruler_propn_component(doc):
#     return ruler_propn(doc)

# Add the components to the pipeline
nlp.add_pipe("ruler_arg", before="ner")
nlp.add_pipe("ruler_argn",before="ner")
nlp.add_pipe("ruler_num", before="ner")
nlp.add_pipe("ruler_adv", before="ner")
nlp.add_pipe("ruler_oper", before="ner")
nlp.add_pipe("ruler_adp", before="ner")
nlp.add_pipe("ruler_noun", before="ner")
nlp.add_pipe("ruler_adj", before="ner")
#nlp.add_pipe("ruler_propn", before = "ner")
ent_formulas,Train_formulas_data=get_entity(df,df.columns[1])

In [29]:
# Test
doc=nlp("SUM (Sepal Length ) AVG ( Sepal Length )")
for ent in doc.ents:
    print(ent.text, ent.label_)
#print(get_entity(df,df.columns[1]))

SUM OPER
Sepal Length ARGN
AVG OPER
Sepal Length ARGN


In [30]:
#Test
print(ent_formulas)

[['SUM(Sepal Length)', ['SUM', 'OPER'], ['Sepal Length', 'ARGN']], ['AVG(Sepal Width)', ['AVG', 'OPER'], ['Sepal Width', 'ARGN']], ['MEDIAN(Petal Length)', ['MEDIAN', 'OPER'], ['Petal Length', 'ARGN']], ['COUNT(*)', ['COUNT', 'OPER']], ['COUNT DISTINCT(Species)', ['COUNT DISTINCT', 'OPER'], ['Species', 'ARGN']], ['MIN(Sepal Length)', ['MIN', 'OPER'], ['Sepal Length', 'ARGN']], ['MAX(Petal Width)', ['MAX', 'OPER'], ['Petal Width', 'ARGN']], ['GROUP BY(Species)', ['GROUP BY', 'OPER'], ['Species', 'ARGN']], ['ORDER BY(Sepal Length ASC)', ['ORDER BY', 'OPER'], ['Sepal Length', 'ARGN'], ['ASC', 'ADV']], ['ORDER BY(Species ASC)', ['ORDER BY', 'OPER'], ['Species', 'ARGN'], ['ASC', 'ADV']], ['Sepal Length >= 5.0', ['Sepal Length', 'ARGN'], ['>=', 'ADP'], ['5.0', 'NUM']], ['Petal Length BETWEEN 1.5 AND 2.5', ['Petal Length', 'ARGN'], ['BETWEEN', 'ADP'], ['1.5', 'NUM'], ['AND', 'ADP'], ['2.5', 'NUM']], ['TOP 3 BY(Sepal Width DESC)', ['TOP', 'OPER'], ['3', 'NUM'], ['BY', 'OPER'], ['Sepal Width', 

#Train Formula (Tableau) data

In [31]:
for item in Train_formulas_data:
    print(item)

('SUM(Sepal Length)', dict_items([('entities', [(1, 2, 'OPER'), (4, 5, 'ARGN'), (10, 11, 'ADP'), (11, 12, 'OPER'), (14, 15, 'ARGN')])]))
('AVG(Sepal Width)', dict_items([('entities', [(1, 2, 'OPER'), (4, 5, 'ARGN'), (10, 11, 'ADP'), (11, 12, 'OPER'), (14, 15, 'ARGN')])]))
('MEDIAN(Petal Length)', dict_items([('entities', [(1, 2, 'OPER'), (4, 5, 'ARGN'), (10, 11, 'ADP'), (11, 12, 'OPER'), (14, 15, 'ARGN')])]))
('COUNT(*)', dict_items([('entities', [(1, 2, 'OPER'), (4, 5, 'ARGN'), (10, 11, 'ADP'), (11, 12, 'OPER'), (14, 15, 'ARGN')])]))
('COUNT DISTINCT(Species)', dict_items([('entities', [(1, 2, 'OPER'), (4, 5, 'ARGN'), (10, 11, 'ADP'), (11, 12, 'OPER'), (14, 15, 'ARGN')])]))
('MIN(Sepal Length)', dict_items([('entities', [(1, 2, 'OPER'), (4, 5, 'ARGN'), (10, 11, 'ADP'), (11, 12, 'OPER'), (14, 15, 'ARGN')])]))
('MAX(Petal Width)', dict_items([('entities', [(1, 2, 'OPER'), (4, 5, 'ARGN'), (10, 11, 'ADP'), (11, 12, 'OPER'), (14, 15, 'ARGN')])]))
('GROUP BY(Species)', dict_items([('entitie

#TRAINING DATA: DESCRIPTION/FORMULAS TABLEAU

In [77]:
def training_data(TRAIN_DATA,NITER,DROP):
    import spacy
    import random

    # Step 1: Load Your Model
    # Load a pre-trained model
    nlp = spacy.load('en_core_web_sm')

    # Or create a blank model
    # nlp = spacy.blank('en')
    # Step 2: Add or Update the NER Pipeline
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe('ner')

    # Add new entity labels to the NER
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
              ner.add_label(ent[2])
    # Step 3: Disable Other Pipeline Components (Optional)
    # Other components in the pipeline, if any
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
          optimizer = nlp.begin_training()
          for itn in range(NITER):  # Number of training iterations
                      random.shuffle(TRAIN_DATA)
                      losses = {}
                      for text, annotations in TRAIN_DATA:
                            nlp.update([text], [annotations], drop=DROP, losses=losses, sgd=optimizer)
                      print(losses)


#SAVE THE TRAINED MODEL

In [76]:
def save_model(model_name,location):
   import os
   from pathlib import Path
   if location=='drive':
      from google.colab import drive
      drive.mount('/content/drive')
      # Define the base path for the folder where you want to save your model
      folder_path = '/content/drive/My Drive/MyModels'
   else:
      # Define the base path for the folder where you want to save your model
      folder_path = '/content/MyModels'

   # Create the folder if it doesn't exist
   os.makedirs(folder_path, exist_ok=True)
   model_path = os.path.join(folder_path, 'model_name')
   # Save the model to the specified directory in Google Drive
   nlp.to_disk(model_path)
   print(f'Model saved to: {model_path}')
   # Load the model to ensure it's been saved correctly
   nlp2 = spacy.load(model_path)



#EVALUATE THE MODEL

In [78]:
def evaluate(ner_model, examples):
    scorer = spacy.scorer.Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = spacy.gold.GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

#Evaluate the Model on the Test Dataset

In [79]:
def evaluate_the_model(path,TEST_DATA):
  import spacy
  nlp = spacy.load(path)
  # Evaluate the model
  results = evaluate(nlp, TEST_DATA)
  return print(results)

# Loading Test Dataset: formulas_test.xlsx

In [ ]:
df=pd.read_excel("formulas.xlsx")
print(df.head(5))

#Extracting symbols from Formulas (Tableau)

In [32]:

def gather_sym(data,column_name):
    dp=data[column_name]
    dp_text=[]
    dp_sym=[]
    for row in dp:
         drow_text=[]
         drow_sym =[]
         for item in row:
              if re.match(r'\w+',item):
                    drow_text.append(item)
              else:
                    drow_sym.append(item)
         dp_text.append(drow_text)
         dp_sym.append(drow_sym)
    return dp_text,dp_sym


In [33]:
dg_formulas_text,dg_formulas_sym=gather_sym(dg_tok,dg_tok.columns[1])

In [34]:
print(dg_formulas_text)

[['SUM', 'Sepal', 'Length'], ['AVG', 'Sepal', 'Width'], ['MEDIAN', 'Petal', 'Length'], ['COUNT'], ['COUNT', 'DISTINCT', 'Species'], ['MIN', 'Sepal', 'Length'], ['MAX', 'Petal', 'Width'], ['GROUP', 'BY', 'Species'], ['ORDER', 'BY', 'Sepal', 'Length', 'ASC'], ['ORDER', 'BY', 'Species', 'ASC'], ['Sepal', 'Length', '5', '0'], ['Petal', 'Length', 'BETWEEN', '1', '5', 'AND', '2', '5'], ['TOP', '3', 'BY', 'Sepal', 'Width', 'DESC'], ['Species', 'Setosa'], ['Species', 'STARTS', 'WITH', 'Se'], ['SUM', 'Petal', 'Length'], ['AVG', 'Petal', 'Width'], ['MEDIAN', 'Sepal', 'Width'], ['COUNT', 'Sepal', 'Length'], ['COUNTD', 'Species'], ['MIN', 'Sepal', 'Length'], ['MAX', 'Petal', 'Width'], ['SORT', 'Species', 'DESC'], ['SORT', 'Sepal', 'Length', 'ASC'], ['Sepal', 'Length', '4', '5'], ['Petal', 'Width', '2', '0'], ['Sepal', 'Width', '3', '0'], ['ENDSWITH', 'Species', 'ica'], ['Sepal', 'Length', '5', '0'], ['INDEX', '5', 'SORT', 'BY', 'SUM', 'Sepal', 'Width', 'DESC'], ['INDEX', '3', 'SORT', 'BY', 'Petal'

In [35]:
print(dg_formulas_sym)

[['(', ')'], ['(', ')'], ['(', ')'], ['(*)'], ['(', ')'], ['(', ')'], ['(', ')'], ['(', ')'], ['(', ')'], ['(', ')'], ['>=', '.'], ['.', '.'], ['(', ')'], ['=', "'", "'"], ["'", "'"], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '])'], ['([', '],', ')'], ['([', '],', ')'], ['[', ']', '>=', '.'], ['[', ']', '<', '.'], ['[', ']', '!=', '.'], ['([', '],', '"', '")'], ['[', ']', '=', '.'], ['()', '<=', '([', '])'], ['()', '<=', '[', ']'], ['[', ']', '>=', '[', ']', '<='], ["('", "',", '[', '])'], ['([', '])', '=', '(', '())', '-'], ['[', ']', '>=', "('", "'", ',', '())'], ['([', '])', '[', ']', '[', ']'], ['(', '([', ']', ')', '=', "'", "'", '[', ']', ')'], ['[', ']', '>', '.', '[', ']', '>', '.'], ['([', '])', '<=', '.'], ['([', '])'], ['([', '])'], ['{', '[', ']', '=', '"', '"', ':', '([', '])', '}'], ['(', '[', ']', '=', '"', '"', ')'], ['(', '[', ']', '>', '.', '[', ']', ')'], ['{', '[', ']', '=', '"', '"', ':', '([', '])', '}'], ['{', '[',

In [36]:
dg_tok.drop(dg_tok.columns[1], axis=1)

,Tokenized Description
0,"[Sum, of, Sepal, Length]"
1,"[Average, Sepal, Width, across, all, entries]"
2,"[Find, the, median, Petal, Length]"
3,"[Count, the, number, of, records, in, the, dat..."
4,"[Count, distinct, species, in, the, dataset]"
5,"[Minimum, value, of, Sepal, Length]"
6,"[Maximum, Petal, Width, found, in, the, dataset]"
7,"[Group, records, by, Species]"
8,"[Sort, the, entries, by, Sepal, Length, in, as..."
9,"[List, Species, alphabetically]"


Adding columns to dg_tok of Formulas and Symbols in separated way

In [37]:
dg_tok["Tokenized Formula (Tableau)"]=dg_formulas_text

In [38]:
dg_tok["Tokenized Symbols (Tableau)"]=dg_formulas_sym

In [39]:
print(dg_tok.head(10))

                               Tokenized Description  \
0                           [Sum, of, Sepal, Length]   
1      [Average, Sepal, Width, across, all, entries]   
2                 [Find, the, median, Petal, Length]   
3  [Count, the, number, of, records, in, the, dat...   
4       [Count, distinct, species, in, the, dataset]   
5                [Minimum, value, of, Sepal, Length]   
6   [Maximum, Petal, Width, found, in, the, dataset]   
7                      [Group, records, by, Species]   
8  [Sort, the, entries, by, Sepal, Length, in, as...   
9                    [List, Species, alphabetically]   

       Tokenized Formula (Tableau) Tokenized Symbols (Tableau)  
0             [SUM, Sepal, Length]                      [(, )]  
1              [AVG, Sepal, Width]                      [(, )]  
2          [MEDIAN, Petal, Length]                      [(, )]  
3                          [COUNT]                       [(*)]  
4       [COUNT, DISTINCT, Species]                      [(

# Recognition of entities

In [40]:
def get_pos_(data,col_name):
  dd=[]
  gg=[]
  for tok in data[col_name]:

          for ent in nlp(str(list(str(tok).split()))):
              # checking text between punct
              dd.append({ent.text:ent.pos_})
              gg.append({ent.pos_:ent.text})
  return dd,gg

In [41]:
def get_patterns(data,col_name):
     dt,dp=get_pos_(data,col_name)

     pos_=[list(item.keys())[0] for item in dp]
     pos_=list(set(pos_))
     print(dt)
     print(col_name,":",pos_)
     patterns={}
     for key in pos_:
         patterns.update({key:list(set([d[key] for d in dp if  key==list(d.keys())[0]]))})
     return patterns

In [42]:
dg_ent={}

In [43]:
p_t=dg_ent['Patterns_Description']=get_patterns(description,"vocab")

[{'[': 'X'}, {"'": 'X'}, {'versicolor': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Maximum': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'from': 'ADP'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'bottom': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'ends': 'VERB'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Calculate': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'List': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NOUN'}, {'1': 'NUM'}, {"'": 'NUM'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NUM'}, {'10': 'NUM'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Lowest': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'Lengths': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'in': 'ADP'}, {"'": 'NOUN'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'or': 'CCONJ'}, {"'": 'PUNCT'}, {']': 'P

In [44]:
for pat in p_d:
   print({pat:p_d[pat]})

{'PRON': ['each', 'there', 'this', 'What', 'all', 'both']}
{'ADP': ['within', 'for', 'above', 'by', 'in', 'of', 'from', 'over', 'with', 'at', 'than', 'between', 'across']}
{'PROPN': ['Group', 'Count', 'Filter', 'Maximum', 'Date', 'Setosa', 'January', 'Median', 'Rank', 'Minimum', 'Iris']}
{'AUX': ['are', 'is']}
{'X': ['setosa', '10th', 'median', 'virginica']}
{'SCONJ': ['where', 'how', 'How']}
{'NUM': ['5', '0', '3', '1', '10', '4', '2']}
{'DET': ['the']}
{'INTJ': ['%']}
{'VERB': ['Find', 'contains', 'starting', 'ends', 'Show', 'found', 'assuming', 'show', 'recorded', 'start', 'starts', 'sorted', 'descending', 'Display', 'ascending', 'Organize', 'count', 'Aggregate', 'grouped']}
{'ADV': ['Sort', 'only', 'alphabetically', 'exactly']}
{'PART': ['not', 'to']}
{'PUNCT': ['?']}
{'NOUN': ['field', 'versicolor', 'Species', 'List', 'Lengths', 'entries', 'Widths', 'observation', 'color', 'value', 'number', 'month', 'Number', 'Entries', 'name', 'year', 'species', 'flowers', 'occurrences', 'Sum', 

In [45]:
p_d=dg_ent['Patterns_Description']=get_patterns(dg_tok,dg_tok.columns[0])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'Sum': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'of': 'ADP'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Average': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'across': 'ADP'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'all': 'PRO

In [46]:
for pat in p_d:
   print({pat:p_d[pat]})

{'PRON': ['each', 'there', 'the', 'a', 'this', 'What', 'all', 'both']}
{'ADP': ['within', 'for', 'of', 'by', 'in', 'above', 'from', 'over', 'with', 'at', 'than', 'between', 'across']}
{'PROPN': ['Sum', "'", 'Width', 'Count', 'Group', 'Species', 'Se', 'Filter', 'Iri', '\\', 'January', 'Median', 'Rank', 'Length', 'Iris']}
{'ADJ': ['Distinct', 'Maximum', 'high', 'bottom', 'middle', 'equal', 'Total', 'Lowest', 'Average', 'Select', 'Highest', 'total', 'Minimum', 'last', 'average', 'smallest', 'least', 'low', 'Petal', 'many', 'less', 'distinct', 'highest', 'greater', 'Top', 'unique']}
{'X': ["'", '[', '0', ']', 'median', 'virginica', 'Length', '1', '10th', '2']}
{'AUX': ['are', "'s", 'is']}
{'SCONJ': ['where', 'how', 'How']}
{'DET': ['a', 'the']}
{'INTJ': ['%']}
{'VERB': ['Find', 'contains', 'starting', 'ends', 'Calculate', 'percentile', 'Show', 'found', 'assuming', 'show', 'top', 'recorded', 'setosa', "'", 'Date', 'start', 'ica', 'starts', 'sorted', 'descending', 'ascending', 'dataset', 'Or

In [47]:
p_t=dg_ent['Patterns_Formula(Tableau)']=get_patterns(formulas,'vocab')

[{'[': 'X'}, {"'": 'PUNCT'}, {'CONTAINS': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'versicolor': 'NOUN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'LEFT': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PROPN'}, {'STARTS': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'FROM': 'ADP'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'INTJ'}, {'-': 'VERB'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'LISTED': 'VERB'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NUM'}, {'9': 'NUM'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'NOUN'}, {'1': 'NUM'}, {"'": 'NUM'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'SORTED': 'ADJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {')': 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'AND': 'CCONJ'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'{': 'PUNCT'}, 

In [48]:
for pat in p_t:
    print({pat:p_t[pat]})

{'PRON': ['}']}
{'ADP': ['BETWEEN', 'FROM', 'WITH']}
{'PROPN': ["'", 'LEFT', 'NULL', 'STARTS', 'DATETRUNC', 'Date', 'Setosa', 'COUNT', 'ASC', 'END', 'SUM', 'MAX', 'GROUP', 'MIN', 'COUNTD', 'AVG', 'MEDIAN']}
{'ADJ': ['highest', 'lowest', 'SORTED']}
{'X': ['CONTAINS', "'", 'STARTSWITH', '<', '=', '[', '>', ']', 'virginica', 'setosa']}
{'AUX': ['RANK_PERCENTILE']}
{'SCONJ': ['IF']}
{'INTJ': ["'"]}
{'VERB': ["'", 'RANK', '-', 'LISTED', 'DESC']}
{'ADV': ['THEN', 'SORT', 'ELSE', 'alphabetically']}
{'PART': ['to']}
{'PUNCT': ["'", ')', '*', '"', ']', '(', '{', '!', ',', '”', ':', '“']}
{'NOUN': ['DISTINCT', "'", 'versicolor', 'Species', 'ORDER', 'TODAY', 'month', 'ENDSWITH', 'YEAR', 'INDEX', 'FIXED', 'color', 'year']}
{'CCONJ': ['AND']}
{'NUM': ["'", '5', 'TOP', '0', '3', '9', '1', '4', '2']}


In [49]:
p_t=dg_ent['Patterns_Formula(Tableau)']=get_patterns(dg_tok,dg_tok.columns[1])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'SUM': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'AVG': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'MEDIAN': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Petal': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'":

In [50]:
for pat in p_t:
    print({pat:p_t[pat]})

{'ADP': ['WITH', 'BY', 'FROM', 'BETWEEN']}
{'PROPN': ['Species', 'LEFT', 'STARTS', 'MIN', 'COUNTD', 'MEDIAN', 'Width', 'MAX', 'GROUP', "'", 'Se', 'DATETRUNC', 'Date', 'COUNT', 'Length', 'ASC', 'AVG', 'Iri', 'Setosa', 'END', 'INDEX']}
{'ADJ': ['highest', 'lowest', 'SORTED', 'Petal']}
{'X': ["'", '[', '0', 'virginica', 'ica', 'Length', '1', '2']}
{'AUX': ['RANK_PERCENTILE']}
{'SCONJ': ['IF']}
{'DET': ['a']}
{'VERB': ['DISTINCT', "'", 'CONTAINS', 'STARTSWITH', 'NULL', 'RANK', 'Date', 'LISTED', 'ENDSWITH', 'DESC', 'SUM', 'setosa', 'color']}
{'ADV': ['THEN', 'SORT', 'ELSE', 'alphabetically']}
{'PART': ['to']}
{'PUNCT': ['"', "'", ']', ',']}
{'NOUN': ["'", 'versicolor', 'Species', 'ORDER', 'TODAY', 'Date', 'month', 'Sepal', 'ENDSWITH', 'C', 'YEAR', 'SUM', 'FIXED', 'year']}
{'CCONJ': ['AND']}
{'NUM': ["'", '5', 'TOP', '3', '9', '4']}


In [51]:
p_e=dg_ent['Patterns_Symbols(Tableau)']=get_patterns(dg_tok,dg_tok.columns[2])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'PROPN'}, {'(': 'PUNCT'}, {'*': 'PUNCT'}, {')': 'PUNCT'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'(': 'PUNCT'}, {"'": 'PUNCT'}, {',

In [52]:
for pat in p_e:
    print({pat:p_e[pat]})

{'PRON': ['}']}
{'PROPN': ['-', "'", '\\', '\\\'"\\']}
{'X': ["'", '<', '=', '[', '>', ']']}
{'VERB': ["'", '=']}
{'PUNCT': ['.', "'", ')', '*', '=', '"', '(', ']', '!', '{', ',', '”', ':', '“']}
{'NOUN': ['\\\'")\\']}
{'NUM': ["'"]}


In [53]:
POS_=[]
for pat in p_d:
   POS_.append(pat)
for pat in p_t:
   POS_.append(pat)
for pat in p_e:
   POS_.append(pat)
POS_=list(set(POS_))
print(POS_, len(POS_))

['PRON', 'NOUN', 'ADP', 'PROPN', 'AUX', 'X', 'DET', 'NUM', 'INTJ', 'VERB', 'ADV', 'PART', 'PUNCT', 'SCONJ', 'CCONJ', 'ADJ'] 16


In [54]:
ENT_={}

for pat in POS_:
  if (p_d.get(pat)!=None and p_t.get(pat)!=None and p_e.get(pat) !=None):
    p_list=p_d[pat]+p_t[pat]+p_e[pat]
  elif (p_d.get(pat)!=None and p_t.get(pat)!=None ):
    p_list=p_d[pat] + p_t[pat]
  else:
    p_list=p_d[pat]

  ENT_.update({pat:[x for x in list(set(p_list))]})

In [55]:
for pat in POS_:
  print ({pat:ENT_[pat]})

{'PRON': ['each', 'there', 'the', 'a', 'this', 'What', 'all', 'both']}
{'NOUN': ['field', 'versicolor', 'Species', 'TODAY', 'List', 'FIXED', 'Lengths', 'entries', 'SUM', 'Widths', 'observation', 'color', "'", 'value', 'number', 'Date', 'month', 'Sepal', 'Number', 'Entries', 'name', '\\\'")\\', 'year', 'species', 'Display', 'occurrences', 'flowers', 'ORDER', 'points', 'Determine', 'types', 'flower', 'records', 'order', 'ENDSWITH', 'C', 'YEAR', 'data']}
{'ADP': ['within', 'for', 'of', 'by', 'in', 'above', 'BY', 'FROM', 'from', 'over', 'with', 'at', 'than', 'WITH', 'between', 'across', 'BETWEEN']}
{'PROPN': ['Species', 'LEFT', 'STARTS', 'January', '-', 'MIN', 'COUNTD', 'Iris', 'MEDIAN', 'Group', 'Width', '\\', 'MAX', 'GROUP', "'", 'Count', 'Filter', 'Se', 'DATETRUNC', 'Date', 'Median', '\\\'"\\', 'COUNT', 'Length', 'ASC', 'AVG', 'Sum', 'Iri', 'Setosa', 'Rank', 'END', 'INDEX']}
{'AUX': ['are', "'s", 'RANK_PERCENTILE', 'is']}
{'X': ["'", '<', '=', '[', '0', ']', '>', 'median', 'virginica', 

#Tokenize with Pre-trained sPacy models

In [56]:
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()

# Description Patterns

In [57]:
dg_ent={}

In [58]:
p_d=dg_ent['Patterns_Description']=get_patterns(dg_tok,dg_tok.columns[0])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'Sum': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'of': 'ADP'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'PUNCT'}, {'Average': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'across': 'ADP'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'all': 'PRO

In [59]:
for pat in p_d:
   print({pat:p_d[pat]})

{'PRON': ['each', 'there', 'the', 'a', 'this', 'What', 'all', 'both']}
{'ADP': ['within', 'for', 'of', 'by', 'in', 'above', 'from', 'over', 'with', 'at', 'than', 'between', 'across']}
{'PROPN': ['Sum', "'", 'Width', 'Count', 'Group', 'Species', 'Se', 'Filter', 'Iri', '\\', 'January', 'Median', 'Rank', 'Length', 'Iris']}
{'ADJ': ['Distinct', 'Maximum', 'high', 'bottom', 'middle', 'equal', 'Total', 'Lowest', 'Average', 'Select', 'Highest', 'total', 'Minimum', 'last', 'average', 'smallest', 'least', 'low', 'Petal', 'many', 'less', 'distinct', 'highest', 'greater', 'Top', 'unique']}
{'X': ["'", '[', '0', ']', 'median', 'virginica', 'Length', '1', '10th', '2']}
{'AUX': ['are', "'s", 'is']}
{'SCONJ': ['where', 'how', 'How']}
{'DET': ['a', 'the']}
{'INTJ': ['%']}
{'VERB': ['Find', 'contains', 'starting', 'ends', 'Calculate', 'percentile', 'Show', 'found', 'assuming', 'show', 'top', 'recorded', 'setosa', "'", 'Date', 'start', 'ica', 'starts', 'sorted', 'descending', 'ascending', 'dataset', 'Or

# Formulas (Tableau) Patterns

In [60]:
p_t=dg_ent['Patterns_Formula(Tableau)']=get_patterns(dg_tok,dg_tok.columns[1])

[{'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'VERB'}, {'SUM': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Length': 'X'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'AVG': 'PROPN'}, {"'": 'NUM'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Sepal': 'NOUN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Width': 'PROPN'}, {"'": 'PUNCT'}, {']': 'PUNCT'}, {'"': 'PUNCT'}, {']': 'PUNCT'}, {'[': 'X'}, {'"': 'PUNCT'}, {'[': 'X'}, {"'": 'X'}, {'MEDIAN': 'PROPN'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'": 'PUNCT'}, {'Petal': 'ADJ'}, {"'": 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {',': 'PUNCT'}, {'"': 'PUNCT'}, {"'":

In [61]:
for pat in p_t:
   print({pat:p_t[pat]})

{'ADP': ['WITH', 'BY', 'FROM', 'BETWEEN']}
{'PROPN': ['Species', 'LEFT', 'STARTS', 'MIN', 'COUNTD', 'MEDIAN', 'Width', 'MAX', 'GROUP', "'", 'Se', 'DATETRUNC', 'Date', 'COUNT', 'Length', 'ASC', 'AVG', 'Iri', 'Setosa', 'END', 'INDEX']}
{'ADJ': ['highest', 'lowest', 'SORTED', 'Petal']}
{'X': ["'", '[', '0', 'virginica', 'ica', 'Length', '1', '2']}
{'AUX': ['RANK_PERCENTILE']}
{'SCONJ': ['IF']}
{'DET': ['a']}
{'VERB': ['DISTINCT', "'", 'CONTAINS', 'STARTSWITH', 'NULL', 'RANK', 'Date', 'LISTED', 'ENDSWITH', 'DESC', 'SUM', 'setosa', 'color']}
{'ADV': ['THEN', 'SORT', 'ELSE', 'alphabetically']}
{'PART': ['to']}
{'PUNCT': ['"', "'", ']', ',']}
{'NOUN': ["'", 'versicolor', 'Species', 'ORDER', 'TODAY', 'Date', 'month', 'Sepal', 'ENDSWITH', 'C', 'YEAR', 'SUM', 'FIXED', 'year']}
{'CCONJ': ['AND']}
{'NUM': ["'", '5', 'TOP', '3', '9', '4']}


#Tutorial of sPacy

In [62]:
text="Sum of Sepal Length SUM (Sepal Length)"
doc=nlp(text)
for token in doc.ents:
   print(token.text, token.label_,token.start_char,token.end_char, token.start, token.ent_id_, token.label, token.vector_norm)
for token in doc:
   print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
           token.shape_, token.is_alpha, token.is_stop,token.ent_iob_, token.ent_type_)

Sepal Length PERSON 25 37 6  380 7.3880568
Sum Sum PROPN NNP nmod Xxx True False O 
of of ADP IN prep xx True True O 
Sepal Sepal PROPN NNP compound Xxxxx True False O 
Length Length PROPN NNP pobj Xxxxx True False O 
SUM SUM PROPN NNP ROOT XXX True False O 
( ( PUNCT -LRB- punct ( False False O 
Sepal Sepal PROPN NNP compound Xxxxx True False B PERSON
Length Length PROPN NNP appos Xxxxx True False I PERSON
) ) PUNCT -RRB- punct ) False False O 


In [63]:
# Create a span for the new entity
fb_ent = span.Span(doc, 0, 1, label="ORG")
orig_ents = list(doc.ents)

In [64]:
# Option 2: Assign a complete list of ents to doc.ents
doc.ents = orig_ents + [fb_ent]

In [65]:
# Add special case rule
from spacy.symbols import ORTH
special_case = [{ORTH: "gim"}, {ORTH: "me"}]
nlp.tokenizer.add_special_case("gimme", special_case)


In [66]:
# Tokenization and POS tagging result
tokens = [("Sum", "VERB"), ("of", "ADP"), ("Sepal", "NOUN"), ("Length", "NOUN")]
# Example mappings based on POS tagging results
operation_mapping = {"VERB": {"Sum": "SUM", "Average": "AVG", "Count": "COUNT"}}
field_mapping = {"NOUN": ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width","Species"]}


#Template of Training data

In [67]:
TRAIN_DATA = []

#for example in your_extracted_data: # see example above entities = []

#for entity in example['data']:  # iterate over the entities


#    text, label, start, end = entity  # ('want', '@command', 2, 6)

#    label = label.split('@')[1].upper()  # not necessary, but nicer

#    end = end - 1  # correct the end character index

#    entities.append((start, end, label))

# add training example of (text, annotations) tuple

#TRAIN_DATA.append((example['sentence'], {'entities': entities}))

This should give you training data that looks like this:

In [68]:


[ ('I want apples', {'entities': [(2, 5, 'COMMAND'), (7, 12, 'FRUIT')]}) ]

[('I want apples', {'entities': [(2, 5, 'COMMAND'), (7, 12, 'FRUIT')]})]

# DEFINE ENTITY RULER BEFORE NER

One potential difficulty in your example is that it's not very close to natural language. The pre-trained English models were trained on ~2m words of general web and news text, so they're not always going to perform perfect out-of-the-box on text with a very different structure.

While you could update the model with more examples of QUANTITY in your specific texts, I think that a rule-based approach might actually be a better and more efficient solution here.

The example in this blog post is actually very close to what you're trying to do:

import spacy

from spacy.pipeline import EntityRuler

nlp = spacy.load("en_core_web_sm")

weights_pattern = [
    {"LIKE_NUM": True},

    {"LOWER": {"IN": ["g", "kg", "grams", "kilograms", "lb", "lbs", "pounds"]}}
]
patterns = [{"label": "QUANTITY", "pattern": weights_pattern}]

ruler = EntityRuler(nlp, patterns=patterns)

nlp.add_pipe(ruler, before="ner")

doc = nlp("U.S. average was 2 lbs.")

print([(ent.text, ent.label_) for ent in doc.ents])

 [('U.S.', 'GPE'), ('2 lbs', 'QUANTITY')]

The statistical named entity recognizer respects pre-defined entities and wil "predict around" them. So if you're adding the EntityRuler before it in the pipeline, your custom QUANTITY entities will be assigned first and will be taken into account when the entity recognizer predicts labels for the remaining tokens.

Note that this example is using the latest version of spaCy, v2.1.x. You might also want to add more patterns to cover different constructions. For more details and inspiration, check out the documentation on the EntityRuler, combining models and rules and the token match pattern syntax.

In [69]:
import spacy

from spacy.pipeline import EntityRuler

nlp = spacy.load("en_core_web_sm")

weights_pattern = [ {"LIKE_NUM": True},
{"LOWER":{"IN":["g","kg","grams","kilograms","lb"
,"lbs","pounds"]}}]

patterns = [{"label": "QUANTITY", "pattern": weights_pattern}]

ruler = EntityRuler(nlp, patterns=patterns,overwrite_ents=True)

ruler.name="entity_ruler"
nlp.add_pipe("entity_ruler", before="ner")

doc = nlp("U.S. average was 2 lbs .")

print([(ent.text, ent.label_) for ent in doc.ents])

[('U.S.', 'GPE'), ('2', 'CARDINAL')]


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/entityruler.py:405: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))


In [70]:
import spacy
from spacy.pipeline import EntityRuler

nlp = spacy.load('en_core_web_sm', disable=['ner'])


patterns=[]
flower_patterns = [{"label": "FLOWER", "pattern": flower} for flower in ["rose", "tulip", "african daisy"]]
patterns.extend(flower_patterns)

# Initialize the EntityRuler for animals
animal_patterns = [{"label": "ANIMAL", "pattern": animal} for animal in ["cat", "dog", "artic fox"]]
patterns.extend(animal_patterns)

# Initialize the EntityRuler and add the combined patterns
ruler = EntityRuler(nlp, patterns=patterns,overwrite_ents=True)
#ruler = EntityRuler(nlp, overwrite_ents=True)
#ruler.add_patterns(patterns)

# Add the EntityRuler to the pipeline with a unique name
ruler.name="entity_ruler"
nlp.add_pipe("entity_ruler", before="ner")

# Example usage
doc = nlp("I saw a rose and a cat.")
for ent in doc.ents:
    print(ent.text, ent.label_)


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/entityruler.py:405: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))


In [71]:
import spacy
from spacy.pipeline import EntityRuler

nlp = spacy.load('en_core_web_sm', disable=['ner'])

# Initialize the EntityRuler
ruler = EntityRuler(nlp, overwrite_ents=True)

# Define patterns
flower_patterns = [{"label": "FLOWER", "pattern": flower} for flower in ["rose", "tulip", "african daisy"]]
animal_patterns = [{"label": "ANIMAL", "pattern": animal} for animal in ["cat", "dog", "artic fox"]]

# Add patterns to the ruler
ruler.add_patterns(flower_patterns + animal_patterns)

ruler.name='entity_ruler'
# Add the EntityRuler to the pipeline
nlp.add_pipe('entity_ruler')

# Test the pipeline
doc = nlp("I saw a rose and a cat.")
for ent in doc.ents:
    print(ent.text, ent.label_)


/usr/local/lib/python3.10/dist-packages/spacy/pipeline/entityruler.py:405: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))


# Claude code !!

In [72]:
import spacy
from spacy.language import Language
from spacy.pipeline import EntityRuler

nlp = spacy.load('en_core_web_sm')

# Function to create and return an EntityRuler with specified patterns
def create_ruler(patterns, label):
    ruler = EntityRuler(nlp, overwrite_ents=True)
    formatted_patterns = [{"label": label, "pattern": pattern} for pattern in patterns]
    ruler.add_patterns(formatted_patterns)
    return ruler

# Define your patterns
flowers = ["rose", "tulip", "african daisy"]
animals = ["cat", "dog", "artic fox"]
weights = ["g","kg","grams","kilograms","lb","lbs","pounds"]



# Create rulers
ruler_plants = create_ruler(flowers, "FLOWER")
ruler_animals = create_ruler(animals, "ANIMAL")
ruler_weights = create_ruler(weights,"QUANTITY")

# Create component factories
@Language.component("ruler_plants")
def ruler_plants_component(doc):
    return ruler_plants(doc)

@Language.component("ruler_animals")
def ruler_animals_component(doc):
    return ruler_animals(doc)

@Language.component('ruler_weights')
def ruler_weights_component(doc):
     return ruler_weights(doc)

# Add the components to the pipeline
nlp.add_pipe("ruler_plants", before="ner")
nlp.add_pipe("ruler_animals", before="ner")
nlp.add_pipe("ruler_weights", before = "ner")

# Test
doc = nlp("I saw a rose and a cat. A dog smells a tulip. U.S. average was 2 lbs ")
for ent in doc.ents:
    print(ent.text, ent.start, ent.end, ent.label_)

rose 3 4 FLOWER
cat 6 7 ANIMAL
dog 9 10 ANIMAL
tulip 12 13 FLOWER
U.S. 14 15 GPE
2 17 18 CARDINAL
lbs 18 19 QUANTITY


from spacy.lang.en import English

from spacy.matcher import Matcher

from spacy.tokens import Span

nlp = English()

matcher = Matcher(nlp.vocab)

def add_event_ent(matcher, doc, i, matches):

    # Get the current match and create tuple of entity label, start and end.

    # Append entity to the doc's entity. (Don't overwrite doc.ents!)

    match_id, start, end = matches[i]

    entity = Span(doc, start, end, label="EVENT")

    doc.ents += (entity,)

    print(entity.text)

pattern = [{"ORTH": "Google"}, {"ORTH": "I"}, {"ORTH": "/"}, {"ORTH": "O"}]

matcher.add("GoogleIO", [pattern], on_match=add_event_ent)

doc = nlp("This is a text about Google I/O")

matches = matcher(doc)

# RULE ENTITY DIRECTLY


# USE OF MACHER

nlp = spacy.load("en_core_web_sm")

matcher = Matcher(nlp.vocab)

Add match ID "HelloWorld" with no callback and one pattern

pattern = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]

matcher.add("HelloWorld", [pattern])

doc = nlp("Hello, world! Hello world!")

matches = matcher(doc)

for match_id, start, end in matches:

    string_id = nlp.vocab.strings[match_id]  # Get string representation

    span = doc[start:end]  # The matched span

    print(match_id, string_id, start, end, span.text)


# VALIDATE ENTITY RULER
ruler = nlp.add_pipe("entity_ruler", config={"validate": True})


# ADD FUNCTION STRATEGY TO ADD A LOT OF NUMBER OF PATTERNS

ruler = nlp.add_pipe("entity_ruler")

patterns = [{"label": "TEST", "pattern": str(i)} for i in range(100000)]

with nlp.select_pipes(enable="tagger"):

    ruler.add_patterns(patterns)

# Span Ruler

In [73]:
import spacy

nlp = spacy.blank("en")
ruler = nlp.add_pipe("span_ruler")
patterns = [{"label": "ORG", "pattern": "Apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]
ruler.add_patterns(patterns)

doc = nlp("Apple is opening its first big office in San Francisco.")
print([(span.text, span.label_) for span in doc.spans["ruler"]])

[('Apple', 'ORG'), ('San Francisco', 'GPE')]


# USAGE OF PRE-TRAINED MODELS

In [74]:
import spacy

nlp = spacy.load("en_core_web_sm")

import en_core_web_sm

nlp = en_core_web_sm.load()

#doc = nlp("This is a sentence.")
doc =nlp("Sum of Sepal Length")

print([(w.text, w.pos_) for w in doc])

# load without NER

nlp = spacy.load("en_core_web_sm", exclude=["ner"])

# source NER from the same pipeline package as the last component

nlp.add_pipe("ner", source=spacy.load("en_core_web_sm"))

# insert the entity ruler

nlp.add_pipe("entity_ruler", before="ner")

[('Sum', 'PROPN'), ('of', 'ADP'), ('Sepal', 'PROPN'), ('Length', 'PROPN')]
